In [45]:
#| default_exp  lib

In [3]:
username = 'Dmitry Grankin'

In [4]:
q =f""" first, provide chain of thought rational on the meeting context, objectioves and participants.

Second, Construct a series of exploratory questions. 
"""

In [5]:
print(q)

Construct a series of exploratory questions based on the meeting's objectives, key discussion points, and the roles of all participants. These questions should be designed to analyze the transcript for strategic insights, focusing on areas such as decision-making processes, implications of these decisions, actionable commitments made during the meeting, and how the topics discussed align with the organizational roles and interests, particularly with regard to Dmitry Grankin's position and responsibilities. The aim is to facilitate a deep understanding of the meeting's outcomes and its relevance to Dmitry Grankin's work, without directly addressing Dmitry Grankin in the questions.


In [13]:

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:

import sys 
sys.path.insert(0,'/app')
sys.path.insert(0,'..')


In [15]:
#| export

from audio.redis import *

import redis
from pathlib import Path
import torch
import torch.nn.functional as F


In [16]:
#| export

def get_inner_redis():
    
    db = redis.from_url(
            "redis://redis:6379",decode_responses=True
        )

    db.ping()

    return db

In [17]:
#| export

def list_directory_contents(path='.'):
    """
    Lists the contents of a directory using pathlib.

    :param path: The directory path to list contents of. Defaults to the current directory.
    :return: None
    """
    p = Path(path)
    return [entry for entry in p.iterdir()]


In [18]:

#| export
def average_embeddings(embeddings):
    """
    Averages embeddings along the second dimension.

    :param embeddings: A 3D tensor of shape (num_slices, num_embeddings, embedding_dim)
                       where num_slices is the number of time slices,
                       num_embeddings is the number of embeddings per time slice,
                       and embedding_dim is the dimensionality of each embedding.
    :return: A 2D tensor of shape (num_slices, embedding_dim) representing the averaged
             embeddings for each time slice.
    """
    return torch.mean(embeddings, dim=1)

def pairwise_cosine_similarity(embeddings):
    """
    Calculate the pairwise cosine similarity of a set of averaged embeddings.

    :param embeddings: A 2D tensor of shape (num_slices, embedding_dim)
                       where num_slices is the number of time slices and
                       embedding_dim is the dimensionality of the averaged embeddings.
    :return: A 2D tensor of shape (num_slices, num_slices) containing
             the pairwise cosine similarity scores between slices.
    """
    embeddings_norm = F.normalize(embeddings, p=2, dim=1)
    similarity_matrix = torch.mm(embeddings_norm, embeddings_norm.t())
    
    return similarity_matrix



In [28]:
#| export
from pathlib import Path
import pandas as pd
import os

def path_info(path: Path = Path.cwd()):
    # Initialize the list to hold file information
    files_info = []

    # Iterate through each file in the given directory
    for file in Path(path).iterdir():
        if file.is_file():  # Ensure we're only including files
            files_info.append({
                "name": file.name,
                "size": file.stat().st_size,
                "created": pd.to_datetime(file.stat().st_ctime, unit='s'),
                "modified": pd.to_datetime(file.stat().st_mtime, unit='s'),
                "path": file  # Add the full path as a Pathlib object
            })

    # Convert the list to a DataFrame and return it
    return pd.DataFrame(files_info)


In [29]:
from audio.audio import AudioSlicer

In [32]:
pathinfo = path_info('/audio').sort_values('created',ascending=False);pathinfo

,name,size,created,modified,path
9,ba149554-1c51-4b70-8e75-9b69a6938331--0.webm,489935,2024-03-08 17:05:48.142953157,2024-03-08 17:05:48.142953157,/audio/ba149554-1c51-4b70-8e75-9b69a6938331--0...
2,7aad9ab0-4a48-47b8-9ccc-d10d718e2d9a--0.webm,1964705,2024-03-08 16:47:13.698168278,2024-03-08 16:47:13.698168278,/audio/7aad9ab0-4a48-47b8-9ccc-d10d718e2d9a--0...
0,41311db9-ed67-4f89-a27f-1d54e2dcd14a--0.webm,1965861,2024-03-08 16:27:08.798985481,2024-03-08 16:27:08.798985481,/audio/41311db9-ed67-4f89-a27f-1d54e2dcd14a--0...
8,39b4bb24-2b9f-4db7-847a-23b48fea6a5d--0.webm,8946592,2024-03-07 17:07:17.538658142,2024-03-07 17:07:17.538658142,/audio/39b4bb24-2b9f-4db7-847a-23b48fea6a5d--0...
3,f4c982ec-3b3f-4044-aad2-e9fb26e44366--0.webm,3669684,2024-03-07 16:54:44.857760191,2024-03-07 16:54:44.857760191,/audio/f4c982ec-3b3f-4044-aad2-e9fb26e44366--0...
1,42381612-e5e2-46ba-bb5a-73d17d25f90f--0.webm,8131419,2024-03-06 18:20:36.538984776,2024-03-06 18:20:36.538984776,/audio/42381612-e5e2-46ba-bb5a-73d17d25f90f--0...
6,a5fc8dae-9463-4973-ae9c-f95ed56b2137--0.webm,43816,2024-03-06 17:47:32.893076658,2024-03-06 17:47:32.893076658,/audio/a5fc8dae-9463-4973-ae9c-f95ed56b2137--0...
4,a72d3780-84f9-4e0a-8a60-6b3aba70bf23--0.webm,12846809,2024-03-06 17:22:50.794866800,2024-03-06 17:22:50.794866800,/audio/a72d3780-84f9-4e0a-8a60-6b3aba70bf23--0...
5,66f2afd0-7482-49ff-964a-c8acf5514379--0.webm,21205097,2024-03-06 13:43:49.819414377,2024-03-06 13:43:49.819414377,/audio/66f2afd0-7482-49ff-964a-c8acf5514379--0...
7,85ecfa13-14ce-4c59-a824-b3405d258f13--0.webm,17004100,2024-03-06 13:21:57.194664955,2024-03-06 13:21:57.194664955,/audio/85ecfa13-14ce-4c59-a824-b3405d258f13--0...


In [3]:
audio_file_path = pathinfo['path'].iloc[0]

NameError: name 'pathinfo' is not defined

In [4]:
call = await AudioSlicer.from_file(audio_file_path,format= 'webm')

NameError: name 'AudioSlicer' is not defined

In [5]:
call.slice(0,10000)

NameError: name 'call' is not defined

In [6]:
8*60

480

In [29]:
160/60

2.6666666666666665